In [23]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

In [24]:
# Read the 4 data files from xlsx
ed = pd.read_excel('data/Education2023.xlsx', skiprows=3) # Education
pop = pd.read_excel('data/PopulationEstimates.xlsx', skiprows=4) # Population
pov = pd.read_excel('data/Poverty2023.xlsx', skiprows=4) # Poverty
un = pd.read_excel('data/Unemployment2023.xlsx', skiprows=4) # Unemployment

In [25]:
# Keep the columns we need
ed_columns_to_keep = [
    "FIPS Code", "State", "Area name",  # Geographic Identifiers
    "2024 Urban Influence Code",
    #"2023 Rural-urban Continuum Code",  # Urbanization & Influence
    # Educational Attainment, 2019-2023
    "Percent of adults who are not high school graduates, 2019-23",
    "Percent of adults who are high school graduates (or equivalent), 2019-23",
    "Percent of adults completing some college or associate degree, 2019-23",
    "Percent of adults with a bachelor's degree or higher, 2019-23",
    # Educational Attainment, 2008-2012
    # "Percent of adults who are not high school graduates, 2008-12",
    # "Percent of adults who are high school graduates (or equivalent), 2008-12",
    # "Percent of adults completing some college or associate degree, 2008-12",
    # "Percent of adults with a bachelor's degree or higher, 2008-12",
    # # Educational Attainment, 2000
    # "Percent of adults who are not high school graduates, 2000",
    # "Percent of adults who are high school graduates (or equivalent), 2000",
    # "Percent of adults completing some college or associate degree, 2000",
    # "Percent of adults with a bachelor's degree or higher, 2000",
    # # Educational Attainment, 1990
    # "Percent of adults who are not high school graduates, 1990",
    # "Percent of adults who are high school graduates (or equivalent), 1990",
    # "Percent of adults completing some college or associate degree, 1990",
    # "Percent of adults with a bachelor's degree or higher, 1990",
    # # Educational Attainment, 1980
    # "Percent of adults with less than a high school diploma, 1980",
    # "Percent of adults with a high school diploma only, 1980",
    # "Percent of adults completing some college (1-3 years), 1980",
    # "Percent of adults completing four years of college or higher, 1980",
    # # Educational Attainment, 1970
    # "Percent of adults with less than a high school diploma, 1970",
    # "Percent of adults with a high school diploma only, 1970",
    # "Percent of adults completing some college (1-3 years), 1970",
    # "Percent of adults completing four years of college or higher, 1970"
]

pov_columns_to_keep = [
    # Geographic Identifiers
    "FIPS_Code",
    #"Stabr", "Area_Name",
    # Urbanization & Influence
    #"Rural_Urban_Continuum_Code_2023", "Urban_Influence_Code_2024",
    # Economic Indicators/Poverty Metrics
    "MEDHHINC_2023", "PCTPOVALL_2023",
    #"PCTPOV017_2023",
    #"PCTPOV517_2023", "PCTPOV04_2023"
]

pop_columns_to_keep = [
    # Geographic Identifiers
    "FIPStxt", "State", "Area_Name",
    # Economic Indicators
    "Economic_typology_2015",
    # Urbanization & Influence
    #"Rural_Urban_Continuum_Code_2023", "Urban_Influence_2013",
    # Population Metrics
    #"CENSUS_2020_POP",
     # "ESTIMATES_BASE_2020",
    # 2020-2023 Metrics
    #"POP_ESTIMATE_2021",
     # "N_POP_CHG_2021",
    #"BIRTHS_2021",
    #"DEATHS_2021",
      "NATURAL_CHG_2023",
    "DOMESTIC_MIG_2023",  "R_BIRTH_2023",
    "R_NATURAL_CHG_2023", "R_INTERNATIONAL_MIG_2023", "R_DOMESTIC_MIG_2023"
    #, "R_NET_MIG_2021"
    #,"INTERNATIONAL_MIG_2021",
    #"NET_MIG_2021",
    #"R_DEATH_2021"
]

un_columns_to_keep = [
    # Geographic Identifiers
    "FIPS_Code",
     # "State", "Area_Name",
    # Economic Indicators
    #"Median_Household_Income_2022"
    #"Med_HH_Income_Percent_of_State_Total_2022",
    # Urbanization & Influence
    #"Rural_Urban_Continuum_Code_2023", "Urban_Influence_Code_2013",
    #"Metro_2023",
    # Employment Metrics over 24 years
    #"Civilian_labor_force_2023",
     "Employed_2023",
      # "Unemployed_2023",
      "Unemployment_rate_2023"
]

ed = ed[ed_columns_to_keep]
pov = pov[pov_columns_to_keep]
pop = pop[pop_columns_to_keep]
un = un[un_columns_to_keep]

In [26]:
# Rename the FIPS column to match
ed.rename(columns={'FIPS Code': 'FIPS'}, inplace=True)
pop.rename(columns={'FIPStxt': 'FIPS'}, inplace=True)
pov.rename(columns={'FIPS_Code': 'FIPS'}, inplace=True)
un.rename(columns={'FIPS_Code': 'FIPS'}, inplace=True)

In [27]:
# Give them names to reference later
ed.name = 'Education'
pop.name = 'Population'
pov.name = 'Poverty'
un.name = 'Unemployment'

In [28]:
# Print the shape of each dataframe
print(f'{ed.name}: {ed.shape}')
print(f'{pop.name}: {pop.shape}')
print(f'{pov.name}: {pov.shape}')
print(f'{un.name}: {un.shape}')

Education: (3294, 8)
Population: (3283, 10)
Poverty: (3196, 3)
Unemployment: (3286, 3)


In [29]:
# Find common FIPS codes
common_fips = set(ed['FIPS']) & set(pop['FIPS']) & set(pov['FIPS']) & set(un['FIPS']) # len(common_fips) = 3195

# Rows that will be dropped during inner merge
ed_lost = ed[~ed['FIPS'].isin(common_fips)]
pop_lost = pop[~pop['FIPS'].isin(common_fips)]
pov_lost = pov[~pov['FIPS'].isin(common_fips)]
un_lost = un[~un['FIPS'].isin(common_fips)]

# Print the number of rows lost for each dataset
print(f'{ed.name}: {len(ed_lost)} rows lost')
print(f'{pop.name}: {len(pop_lost)} rows lost')
print(f'{pov.name}: {len(pov_lost)} rows lost')
print(f'{un.name}: {len(un_lost)} rows lost')
# Overseas territories and a few domestic territories are lost

Education: 99 rows lost
Population: 88 rows lost
Poverty: 1 rows lost
Unemployment: 91 rows lost


In [30]:
ed_lost.to_csv('data/ed_lost.csv', index=False)

In [31]:
# Merge the dataframes
merged = ed.merge(pop, on='FIPS').merge(pov, on='FIPS').merge(un, on='FIPS')

print(f'Merged shape: {merged.shape}')

Merged shape: (3195, 21)


In [32]:
# Check for missing values
missing = merged.isnull().sum()
print(missing[missing > 0])

2024 Urban Influence Code    52
Economic_typology_2015       63
Employed_2023                 9
Unemployment_rate_2023        9
dtype: int64


In [33]:
# Remove rows with missing values
merged = merged.dropna()

In [34]:
# Remove 'State_y' column and rename 'State_x' to 'State'
merged.drop(columns='State_y', inplace=True)
merged.rename(columns={'State_x': 'State'}, inplace=True)

In [35]:
merged

,FIPS,State,Area name,2024 Urban Influence Code,"Percent of adults who are not high school graduates, 2019-23","Percent of adults who are high school graduates (or equivalent), 2019-23","Percent of adults completing some college or associate degree, 2019-23","Percent of adults with a bachelor's degree or higher, 2019-23",Area_Name,Economic_typology_2015,NATURAL_CHG_2023,DOMESTIC_MIG_2023,R_BIRTH_2023,R_NATURAL_CHG_2023,R_INTERNATIONAL_MIG_2023,R_DOMESTIC_MIG_2023,MEDHHINC_2023,PCTPOVALL_2023,Employed_2023,Unemployment_rate_2023
2,1001,AL,Autauga County,4.0,9.721098,32.675939,29.320284,28.282680,Autauga County,0.0,93.0,491.0,11.9,1.5,0.6,8.2,68857.0,11.7,26409.0,2.2
3,1003,AL,Baldwin County,4.0,8.268600,27.542038,31.391725,32.797637,Baldwin County,5.0,-109.0,6804.0,10.1,-0.4,1.2,27.2,74248.0,10.0,102029.0,2.3
4,1005,AL,Barbour County,5.0,22.186295,37.604947,28.744044,11.464715,Barbour County,3.0,-83.0,-45.0,10.8,-3.4,0.5,-1.8,45298.0,25.5,7592.0,4.4
5,1007,AL,Bibb County,1.0,19.659783,41.127362,27.744649,11.468207,Bibb County,0.0,-50.0,-69.0,10.9,-2.3,0.0,-3.1,56025.0,19.4,8556.0,2.5
6,1009,AL,Blount County,1.0,17.303798,35.617575,31.499597,15.579030,Blount County,0.0,-78.0,358.0,11.7,-1.3,0.4,6.0,64962.0,12.8,25282.0,2.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3190,56037,WY,Sweetwater County,7.0,7.468835,31.147245,41.008130,20.375790,Sweetwater County,2.0,64.0,-180.0,10.4,1.5,-0.3,-4.4,76442.0,10.0,19575.0,3.3
3191,56039,WY,Teton County,7.0,2.840434,16.145920,19.793929,61.219716,Teton County,5.0,100.0,-221.0,8.2,4.3,2.4,-9.5,130156.0,6.2,16955.0,2.1
3192,56041,WY,Uinta County,7.0,5.393292,35.990140,37.297378,21.319190,Uinta County,2.0,47.0,-23.0,11.0,2.3,-0.3,-1.1,79086.0,10.3,8807.0,3.2
3193,56043,WY,Washakie County,9.0,8.580258,30.267224,39.120160,22.032358,Washakie County,0.0,-37.0,25.0,9.3,-4.8,-0.1,3.2,62794.0,12.3,3716.0,3.3


In [36]:
# Remove 'Area Name' and 'Area_Name' columns
merged.drop(columns=['Area name', 'Area_Name'], inplace=True)

In [38]:
merged

,FIPS,State,2024 Urban Influence Code,"Percent of adults who are not high school graduates, 2019-23","Percent of adults who are high school graduates (or equivalent), 2019-23","Percent of adults completing some college or associate degree, 2019-23","Percent of adults with a bachelor's degree or higher, 2019-23",Economic_typology_2015,NATURAL_CHG_2023,DOMESTIC_MIG_2023,R_BIRTH_2023,R_NATURAL_CHG_2023,R_INTERNATIONAL_MIG_2023,R_DOMESTIC_MIG_2023,MEDHHINC_2023,PCTPOVALL_2023,Employed_2023,Unemployment_rate_2023
2,1001,AL,4.0,9.721098,32.675939,29.320284,28.282680,0.0,93.0,491.0,11.9,1.5,0.6,8.2,68857.0,11.7,26409.0,2.2
3,1003,AL,4.0,8.268600,27.542038,31.391725,32.797637,5.0,-109.0,6804.0,10.1,-0.4,1.2,27.2,74248.0,10.0,102029.0,2.3
4,1005,AL,5.0,22.186295,37.604947,28.744044,11.464715,3.0,-83.0,-45.0,10.8,-3.4,0.5,-1.8,45298.0,25.5,7592.0,4.4
5,1007,AL,1.0,19.659783,41.127362,27.744649,11.468207,0.0,-50.0,-69.0,10.9,-2.3,0.0,-3.1,56025.0,19.4,8556.0,2.5
6,1009,AL,1.0,17.303798,35.617575,31.499597,15.579030,0.0,-78.0,358.0,11.7,-1.3,0.4,6.0,64962.0,12.8,25282.0,2.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3190,56037,WY,7.0,7.468835,31.147245,41.008130,20.375790,2.0,64.0,-180.0,10.4,1.5,-0.3,-4.4,76442.0,10.0,19575.0,3.3
3191,56039,WY,7.0,2.840434,16.145920,19.793929,61.219716,5.0,100.0,-221.0,8.2,4.3,2.4,-9.5,130156.0,6.2,16955.0,2.1
3192,56041,WY,7.0,5.393292,35.990140,37.297378,21.319190,2.0,47.0,-23.0,11.0,2.3,-0.3,-1.1,79086.0,10.3,8807.0,3.2
3193,56043,WY,9.0,8.580258,30.267224,39.120160,22.032358,0.0,-37.0,25.0,9.3,-4.8,-0.1,3.2,62794.0,12.3,3716.0,3.3


In [37]:
# List all columns and their data types
merged.dtypes

FIPS                                                                          int64
State                                                                        object
2024 Urban Influence Code                                                   float64
Percent of adults who are not high school graduates, 2019-23                float64
Percent of adults who are high school graduates (or equivalent), 2019-23    float64
Percent of adults completing some college or associate degree, 2019-23      float64
Percent of adults with a bachelor's degree or higher, 2019-23               float64
Economic_typology_2015                                                      float64
NATURAL_CHG_2023                                                            float64
DOMESTIC_MIG_2023                                                           float64
R_BIRTH_2023                                                                float64
R_NATURAL_CHG_2023                                                          

In [39]:
# Normalize the numeric columns
numeric_columns = merged.columns[2:]
scaler = StandardScaler()
merged[numeric_columns] = scaler.fit_transform(merged[numeric_columns])

In [40]:
merged

,FIPS,State,2024 Urban Influence Code,"Percent of adults who are not high school graduates, 2019-23","Percent of adults who are high school graduates (or equivalent), 2019-23","Percent of adults completing some college or associate degree, 2019-23","Percent of adults with a bachelor's degree or higher, 2019-23",Economic_typology_2015,NATURAL_CHG_2023,DOMESTIC_MIG_2023,R_BIRTH_2023,R_NATURAL_CHG_2023,R_INTERNATIONAL_MIG_2023,R_DOMESTIC_MIG_2023,MEDHHINC_2023,PCTPOVALL_2023,Employed_2023,Unemployment_rate_2023
2,1001,AL,-0.426905,-0.278537,-0.151198,-0.297658,0.418089,-0.995489,-0.051421,0.125382,0.480823,0.781117,-0.333898,0.359741,0.205146,-0.505364,-0.147065,-1.137550
3,1003,AL,-0.426905,-0.543725,-0.822225,0.096189,0.862459,1.754927,-0.205564,1.747120,-0.246254,0.374123,-0.052500,1.866632,0.532121,-0.811439,0.310732,-1.055631
4,1005,AL,-0.037317,1.997280,0.493048,-0.407219,-1.237164,0.654761,-0.185724,-0.012310,0.036498,-0.268497,-0.380797,-0.433359,-1.223756,1.979240,-0.260981,0.664661
5,1007,AL,-1.595670,1.536006,0.953446,-0.597235,-1.236820,-0.995489,-0.160542,-0.018475,0.076891,-0.032870,-0.615296,-0.536462,-0.573141,0.880973,-0.255145,-0.891794
6,1009,AL,-1.595670,1.105865,0.233289,0.116699,-0.832226,-0.995489,-0.181909,0.091216,0.400036,0.181337,-0.427697,0.185259,-0.031094,-0.307316,-0.153887,-1.219469
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3190,56037,WY,0.741860,-0.689741,-0.351006,1.924569,-0.360120,0.104677,-0.073551,-0.046990,-0.125074,0.781117,-0.755994,-0.639565,0.665192,-0.811439,-0.188437,-0.236445
3191,56039,WY,0.741860,-1.534765,-2.311755,-2.108917,3.659809,1.754927,-0.046079,-0.057523,-1.013723,1.380896,0.510296,-1.044046,3.923056,-1.495605,-0.204298,-1.219469
3192,56041,WY,0.741860,-1.068681,0.281985,1.219039,-0.267269,0.104677,-0.086523,-0.006659,0.117285,0.952482,-0.755994,-0.377842,0.825556,-0.757425,-0.253626,-0.318363
3193,56043,WY,1.521036,-0.486824,-0.466029,1.565607,-0.197078,-0.995489,-0.150622,0.005672,-0.569399,-0.568387,-0.662195,-0.036809,-0.162588,-0.397338,-0.284446,-0.236445


In [41]:
# One-hot encode the 'State' column
merged_encoded = pd.get_dummies(merged, columns=['State'])

In [42]:
merged_encoded

,FIPS,2024 Urban Influence Code,"Percent of adults who are not high school graduates, 2019-23","Percent of adults who are high school graduates (or equivalent), 2019-23","Percent of adults completing some college or associate degree, 2019-23","Percent of adults with a bachelor's degree or higher, 2019-23",Economic_typology_2015,NATURAL_CHG_2023,DOMESTIC_MIG_2023,R_BIRTH_2023,...,State_SD,State_TN,State_TX,State_UT,State_VA,State_VT,State_WA,State_WI,State_WV,State_WY
2,1001,-0.426905,-0.278537,-0.151198,-0.297658,0.418089,-0.995489,-0.051421,0.125382,0.480823,...,False,False,False,False,False,False,False,False,False,False
3,1003,-0.426905,-0.543725,-0.822225,0.096189,0.862459,1.754927,-0.205564,1.747120,-0.246254,...,False,False,False,False,False,False,False,False,False,False
4,1005,-0.037317,1.997280,0.493048,-0.407219,-1.237164,0.654761,-0.185724,-0.012310,0.036498,...,False,False,False,False,False,False,False,False,False,False
5,1007,-1.595670,1.536006,0.953446,-0.597235,-1.236820,-0.995489,-0.160542,-0.018475,0.076891,...,False,False,False,False,False,False,False,False,False,False
6,1009,-1.595670,1.105865,0.233289,0.116699,-0.832226,-0.995489,-0.181909,0.091216,0.400036,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3190,56037,0.741860,-0.689741,-0.351006,1.924569,-0.360120,0.104677,-0.073551,-0.046990,-0.125074,...,False,False,False,False,False,False,False,False,False,True
3191,56039,0.741860,-1.534765,-2.311755,-2.108917,3.659809,1.754927,-0.046079,-0.057523,-1.013723,...,False,False,False,False,False,False,False,False,False,True
3192,56041,0.741860,-1.068681,0.281985,1.219039,-0.267269,0.104677,-0.086523,-0.006659,0.117285,...,False,False,False,False,False,False,False,False,False,True
3193,56043,1.521036,-0.486824,-0.466029,1.565607,-0.197078,-0.995489,-0.150622,0.005672,-0.569399,...,False,False,False,False,False,False,False,False,False,True


In [43]:
# Print the shape of df before and after one-hot encoding
print(f'Merged shape before encoding: {merged.shape}')
print(f'Merged shape after encoding: {merged_encoded.shape}')

Merged shape before encoding: (3132, 18)
Merged shape after encoding: (3132, 67)


In [45]:
merged_encoded.columns

Index(['FIPS', '2024 Urban Influence Code',
       'Percent of adults who are not high school graduates, 2019-23',
       'Percent of adults who are high school graduates (or equivalent), 2019-23',
       'Percent of adults completing some college or associate degree, 2019-23',
       'Percent of adults with a bachelor's degree or higher, 2019-23',
       'Economic_typology_2015', 'NATURAL_CHG_2023', 'DOMESTIC_MIG_2023',
       'R_BIRTH_2023', 'R_NATURAL_CHG_2023', 'R_INTERNATIONAL_MIG_2023',
       'R_DOMESTIC_MIG_2023', 'MEDHHINC_2023', 'PCTPOVALL_2023',
       'Employed_2023', 'Unemployment_rate_2023', 'State_AK', 'State_AL',
       'State_AR', 'State_AZ', 'State_CA', 'State_CO', 'State_DC', 'State_DE',
       'State_FL', 'State_GA', 'State_HI', 'State_IA', 'State_ID', 'State_IL',
       'State_IN', 'State_KS', 'State_KY', 'State_LA', 'State_MA', 'State_MD',
       'State_ME', 'State_MI', 'State_MN', 'State_MO', 'State_MS', 'State_MT',
       'State_NC', 'State_ND', 'State_NE', 'Sta

In [46]:
merged_encoded.dtypes

FIPS                                                                          int64
2024 Urban Influence Code                                                   float64
Percent of adults who are not high school graduates, 2019-23                float64
Percent of adults who are high school graduates (or equivalent), 2019-23    float64
Percent of adults completing some college or associate degree, 2019-23      float64
                                                                             ...   
State_VT                                                                       bool
State_WA                                                                       bool
State_WI                                                                       bool
State_WV                                                                       bool
State_WY                                                                       bool
Length: 67, dtype: object